In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv('Churn_Modelling.csv')

In [8]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
# Drop irrelevant columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [10]:
# Convert the categorical variables to numerical values
label_encoder = LabelEncoder()

# Apply Label Encoding on 'Gender' and 'Geography'
df['Gender'] = label_encoder.fit_transform(df['Gender'])  # Female=0, Male=1
df['Geography'] = label_encoder.fit_transform(df['Geography'])  # France=0, Germany=1, Spain=2

In [11]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,772,1,1,42,3,75075.31,2,1,0,92888.52,1


In [12]:
# Separate the features and the target variables
X = df.drop('Exited', axis = 1) # Features
Y = df['Exited'] # Target

In [14]:
# Split the dataset into training and testing set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [15]:
# Normalize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)
# we use only transform on xtest as it uses the same mean and SD learned from the training set

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [20]:

model = Sequential()
model.add(Dense(activation='relu', units=32, input_dim=X_train.shape[1]))  # Input layer with more neurons
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(activation='relu', units=16))  # Second hidden layer
model.add(Dense(activation='sigmoid', units=1))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, batch_size=32, epochs=50)

Epoch 1/50
250/250 [==============================] - 0s 1ms/step - loss: 0.5531 - accuracy: 0.7320
Epoch 2/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4638 - accuracy: 0.7962
Epoch 3/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4404 - accuracy: 0.8084
Epoch 4/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4275 - accuracy: 0.8164
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4134 - accuracy: 0.8230
Epoch 6/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4071 - accuracy: 0.8301
Epoch 7/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4000 - accuracy: 0.8346
Epoch 8/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3886 - accuracy: 0.8367
Epoch 9/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3862 - accuracy: 0.8375
Epoch 10/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3834 - accuracy: 0.8391

In [23]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5) # to convert the probabilistic values to True or False
y_pred

array([[False],
       [False],
       [False],
       ...,
       [ True],
       [False],
       [ True]])

In [25]:
from sklearn.metrics import accuracy_score, confusion_matrix
# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)

# Print confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the results
print("Accuracy Score:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

Accuracy Score: 0.841
Confusion Matrix:
 [[1540   27]
 [ 291  142]]
